In [2]:
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np

import tensorflow
import time 
import warnings 
import numpy as np 
from numpy import newaxis 
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM 
from keras.models import Sequential 
from sklearn.preprocessing import MinMaxScaler


import time
import math
import matplotlib.pyplot as plt
from keras.models import load_model

import argparse

import helpers
import pickle

Using TensorFlow backend.


In [5]:
f = pd.read_csv("predicted_AQI2019-03-12.csv")
f

,County,State,state_county,County Code,date,AQI


In [ ]:
data2019_raw = pd.read_csv("../../Data/Air_Pollution/data_air_raw/daily_aqi_by_county_2019.csv")
data2019 = helpers.data_cleaning(data2019_raw)

## initialization 
predicted_AQI = []
predicted_date = "2019-03-12"

# for county in list(data2019["state_county"].unique()):
for county in ["binhthuan","wyoming_sublette"]:

    
    ## load model to predict AQI
    print("---> Loading model for county {} ...".format(county))

    try:
        scaler_path = "../../Trained_model/MinMax_scaler_model/" + county + "_scaler.pickle"
        model_path = "../../Trained_model/county_AQI_model/" + county + "_model.h5"
        
        model = load_model(model_path)
        mm_scaler = pickle.load(open( scaler_path, "rb" ))
        
    except Exception as e:
        e.args += ('Path and list_year must not be empty', "check read_raw_data function" )
        continue 
#         raise e
    print("here")
    data_feature_temp = helpers.data_feature_engineering_for_test(data2019, county, predicted_date)

---> Loading model for county binhthuan ...
---> Loading model for county wyoming_sublette ...
Instructions for updating:
Colocations handled automatically by placer.


In [27]:
list(DATA["state_county"].unique())[:1]

['alabama_baldwin']

In [31]:
df_state = DATA[DATA["state_county"] == "alabama_baldwin"]
df_state["date"] = df_state["Date"].apply(lambda x: pd.to_datetime(x))
df_state["current_date"] = df_state["date"].dt.day
df_state["current_month"] = df_state["date"].apply(lambda x: x.month)
df_state['day_of_week'] = df_state['date'].dt.weekday_name

day_df = pd.get_dummies(df_state["day_of_week"], prefix="day")
df_temp = pd.concat([df_state, day_df], axis = 1)

df_feature = df_temp[list(day_df.columns) + ["AQI", "current_date", "current_month", "date"]]
df_feature = df_feature.sort_values(by=["date"])
df_lag_features = compute_lag_time_series_features(df_feature)



In [40]:
df_lag_features

,AQI,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_20,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29
29,33.0,22.0,28.0,13.0,36.0,23.0,26.0,20.0,31.0,28.0,...,23.0,36.0,39.0,36.0,44.0,44.0,45.0,42.0,38.0,41
30,22.0,28.0,13.0,36.0,23.0,26.0,20.0,31.0,28.0,20.0,...,36.0,39.0,36.0,44.0,44.0,45.0,42.0,38.0,41.0,30
31,28.0,13.0,36.0,23.0,26.0,20.0,31.0,28.0,20.0,29.0,...,39.0,36.0,44.0,44.0,45.0,42.0,38.0,41.0,30.0,29
32,13.0,36.0,23.0,26.0,20.0,31.0,28.0,20.0,29.0,20.0,...,36.0,44.0,44.0,45.0,42.0,38.0,41.0,30.0,29.0,46
33,36.0,23.0,26.0,20.0,31.0,28.0,20.0,29.0,20.0,11.0,...,44.0,44.0,45.0,42.0,38.0,41.0,30.0,29.0,46.0,43
34,23.0,26.0,20.0,31.0,28.0,20.0,29.0,20.0,11.0,45.0,...,44.0,45.0,42.0,38.0,41.0,30.0,29.0,46.0,43.0,40
35,26.0,20.0,31.0,28.0,20.0,29.0,20.0,11.0,45.0,22.0,...,45.0,42.0,38.0,41.0,30.0,29.0,46.0,43.0,40.0,41
36,20.0,31.0,28.0,20.0,29.0,20.0,11.0,45.0,22.0,32.0,...,42.0,38.0,41.0,30.0,29.0,46.0,43.0,40.0,41.0,39
37,31.0,28.0,20.0,29.0,20.0,11.0,45.0,22.0,32.0,25.0,...,38.0,41.0,30.0,29.0,46.0,43.0,40.0,41.0,39.0,34
38,28.0,20.0,29.0,20.0,11.0,45.0,22.0,32.0,25.0,10.0,...,41.0,30.0,29.0,46.0,43.0,40.0,41.0,39.0,34.0,35


In [30]:
def feature_engineering_for_AQI(data, lag_time = 30, county_name = "", save_path = ""):
    """
    test: make sure save_path is not empty, data row > 10, data have enough columns, 
    """
    
    try:
        df_state = data[data["state_county"] == county_name]
    except:
        raise AttributeError("DATAFRAME IS EMPTY !!!!!")

    try:
        df_state["date"] = df_state["Date"].apply(lambda x: pd.to_datetime(x))
        df_state["current_date"] = df_state["date"].dt.day
        df_state["current_month"] = df_state["date"].apply(lambda x: x.month)
        df_state['day_of_week'] = df_state['date'].dt.weekday_name

        day_df = pd.get_dummies(df_state["day_of_week"], prefix="day")
        df_temp = pd.concat([df_state, day_df], axis = 1)

        df_feature = df_temp[list(day_df.columns) + ["AQI", "current_date", "current_month", "date"]]
        df_feature = df_feature.sort_values(by=["date"])

        df_lag_features = compute_lag_time_series_features(df_feature)
        row = np.min([df_feature.shape[0]-1, lag_time-1])
        df_data = (pd.concat([df_lag_features.drop(["AQI"], axis = 1), 
                         df_feature.drop(["date"], axis = 1).iloc[row:,:]], axis = 1))

        if save_path: 
            path = save_path + county_name + "_feature.csv"
            print("---> Saving features to {}".format(path))
            df_data.to_csv(path, index = False)
            
    except:
        raise AttributeError("DATAFRAME IS EMPTY !!!!!")

    return {"successive code": 1, "save_path": save_path, "feature_names": df_data.columns, "data": df_data}

def compute_lag_time_series_features(df_feature, lag_time = 30):
    
    assert df_feature.shape[0] >= 1
    
    try:
        temps = df_feature.sort_values(by=["date"])["AQI"]
        dataframe = temps
        col_name=['AQI']
        for lag_index in range(1,lag_time):
            dataframe = concat([temps.shift(lag_index), dataframe], axis=1)
            col_name.append('lag_' + str(lag_index))
        dataframe.columns = col_name

        if dataframe.shape[0] < lag_time:
            lag_features = dataframe.iloc[-1:,:]
            lag_features = lag_features.fillna(0)
        else:
            lag_features = dataframe.iloc[lag_time-1:,:]
    except:
        raise AttributeError("FEATURE DATAFRAME IS EMPTY !!!!!")
   
    return lag_features

In [11]:
y_test.shape


(205,)

In [12]:
scaler

MinMaxScaler(copy=True, feature_range=(0, 1))